In [ ]:
# %% [markdown]
# ## Analiza skupień dla Breast Cancer Wisconsin
# 
# **Autor:** [Twoje Imię]  
# **Data:** [Data]  
# 
# ### Cel projektu
# Porównanie trzech metod klasteryzacji (K-means, DBSCAN, Agglomerative Clustering) na danych Breast Cancer Wisconsin.

# %%
# Krok 1: Import niezbędnych bibliotek
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage

%matplotlib inline

# %%
# Krok 2: Wczytanie i przygotowanie danych
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = data.feature_names
target_names = data.target_names

print(f"Liczba próbek: {X.shape[0]}")
print(f"Liczba cech: {X.shape[1]}")
print(f"Klasy: {np.unique(y)} -> {target_names}")

# Standaryzacja danych
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Redukcja wymiarów do wizualizacji
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Wizualizacja danych oryginalnych
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y, palette='viridis', s=50)
plt.title('PCA: Rzeczywiste klasy')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.legend(title='Klasa', labels=target_names)
plt.show()

# %%
# Krok 3: K-means z różną liczbą skupień
k_values = range(2, 11)
silhouette_scores_kmeans = []
ari_scores_kmeans = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    cluster_labels = kmeans.fit_predict(X_scaled)
    
    # Obliczenie silhouette score
    sil_score = silhouette_score(X_scaled, cluster_labels)
    silhouette_scores_kmeans.append(sil_score)
    
    # Obliczenie Adjusted Rand Index (porównanie z prawdziwymi etykietami)
    ari = adjusted_rand_score(y, cluster_labels)
    ari_scores_kmeans.append(ari)

# Wizualizacja wyników
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(k_values, silhouette_scores_kmeans, 'bo-')
plt.xlabel('Liczba skupień (k)')
plt.ylabel('Silhouette Score')
plt.title('K-means: Współczynnik Silhouette')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(k_values, ari_scores_kmeans, 'ro-')
plt.xlabel('Liczba skupień (k)')
plt.ylabel('Adjusted Rand Index')
plt.title('K-means: Dopasowanie do rzeczywistych klas')
plt.grid(True)

plt.tight_layout()
plt.show()

# %%
# Krok 4: DBSCAN z różnymi parametrami eps i min_samples
eps_values = np.linspace(0.1, 1.0, 10)
min_samples_values = [5, 10, 15, 20]
results = []

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        cluster_labels = dbscan.fit_predict(X_scaled)
        
        # Pomijamy przypadki z tylko 1 klastrem
        unique_labels = np.unique(cluster_labels)
        n_clusters = len(unique_labels) - (1 if -1 in cluster_labels else 0)
        
        if n_clusters < 2:
            continue
            
        sil_score = silhouette_score(X_scaled, cluster_labels)
        ari = adjusted_rand_score(y, cluster_labels)
        noise_ratio = np.sum(cluster_labels == -1) / len(cluster_labels)
        
        results.append({
            'eps': eps,
            'min_samples': min_samples,
            'silhouette': sil_score,
            'ari': ari,
            'n_clusters': n_clusters,
            'noise_ratio': noise_ratio
        })

# Konwersja wyników do DataFrame
results_df = pd.DataFrame(results)

# Wizualizacja wyników
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
sns.heatmap(results_df.pivot_table(index='eps', columns='min_samples', values='silhouette'), 
            annot=True, cmap='viridis', fmt=".3f")
plt.title('DBSCAN: Silhouette Score')
plt.xlabel('min_samples')
plt.ylabel('eps')

plt.subplot(2, 1, 2)
sns.heatmap(results_df.pivot_table(index='eps', columns='min_samples', values='ari'), 
            annot=True, cmap='plasma', fmt=".3f")
plt.title('DBSCAN: Adjusted Rand Index')
plt.xlabel('min_samples')
plt.ylabel('eps')

plt.tight_layout()
plt.show()

# %%
# Krok 5: Agglomerative Clustering
# Hierarchiczna wizualizacja
plt.figure(figsize=(12, 8))
linked = linkage(X_scaled, 'ward')
dendrogram(linked, truncate_mode='level', p=5)
plt.title('Dendrogram')
plt.xlabel('Indeks próbki')
plt.ylabel('Odległość')
plt.show()

# %%
# Krok 6: Porównanie wszystkich trzech metod
# Wybór najlepszych parametrów
best_k = k_values[np.argmax(silhouette_scores_kmeans)]
print(f"Optymalna liczba klastrów dla K-means: {best_k}")

best_dbscan_row = results_df.loc[results_df['silhouette'].idxmax()]
print(f"Optymalne parametry DBSCAN: eps={best_dbscan_row['eps']:.2f}, min_samples={best_dbscan_row['min_samples']}")

# Klasteryzacja z optymalnymi parametrami
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init='auto')
kmeans_labels = kmeans.fit_predict(X_scaled)

dbscan = DBSCAN(eps=best_dbscan_row['eps'], min_samples=int(best_dbscan_row['min_samples']))
dbscan_labels = dbscan.fit_predict(X_scaled)

agg = AgglomerativeClustering(n_clusters=best_k)
agg_labels = agg.fit_predict(X_scaled)

# Obliczenie metryk
metrics = []
for name, labels in zip(['K-means', 'DBSCAN', 'Agglomerative'],
                       [kmeans_labels, dbscan_labels, agg_labels]):
    unique_labels = np.unique(labels)
    n_clusters = len(unique_labels) - (1 if -1 in labels else 0)
    
    if n_clusters > 1:  # silhouette_score wymaga >1 klastra
        sil = silhouette_score(X_scaled, labels)
    else:
        sil = -1
        
    ari = adjusted_rand_score(y, labels)
    noise_ratio = np.sum(labels == -1) / len(labels) if -1 in labels else 0
    
    metrics.append({
        'Metoda': name,
        'Liczba klastrów': n_clusters,
        'Silhouette': sil,
        'Adjusted Rand Index': ari,
        'Procent szumu (%)': noise_ratio * 100
    })

metrics_df = pd.DataFrame(metrics)

# Wizualizacja wyników
plt.figure(figsize=(18, 5))

for i, (name, labels) in enumerate(zip(['K-means', 'DBSCAN', 'Agglomerative'],
                                     [kmeans_labels, dbscan_labels, agg_labels]), 1):
    plt.subplot(1, 3, i)
    scatter = sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], 
                    hue=labels, palette='viridis', 
                    style=y, s=70, alpha=0.8)
    plt.title(f'{name}\nLiczba klastrów: {metrics_df.iloc[i-1]["Liczba klastrów"]}')
    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')
    
    # Dodanie legendy tylko dla pierwszego wykresu
    if i == 1:
        handles, _ = scatter.get_legend_handles_labels()
        plt.legend(handles, ['Klaster', 'Klasy (kształt)'], loc='best')
    else:
        plt.legend([], [], frameon=False)

plt.tight_layout()
plt.show()

# Wyświetlenie tabeli z metrykami
display(metrics_df)

# %%
# Krok 7: Analiza wyników
print("\nPodsumowanie wyników:")
print("1. K-means:")
print(f"   - Optymalna liczba klastrów: {best_k}")
print(f"   - Silhouette: {metrics_df[metrics_df['Metoda']=='K-means']['Silhouette'].values[0]:.4f}")
print(f"   - ARI: {metrics_df[metrics_df['Metoda']=='K-means']['Adjusted Rand Index'].values[0]:.4f}")

print("\n2. DBSCAN:")
print(f"   - Najlepsze parametry: eps={best_dbscan_row['eps']:.2f}, min_samples={best_dbscan_row['min_samples']}")
print(f"   - Liczba klastrów: {metrics_df[metrics_df['Metoda']=='DBSCAN']['Liczba klastrów'].values[0]}")
print(f"   - Silhouette: {metrics_df[metrics_df['Metoda']=='DBSCAN']['Silhouette'].values[0]:.4f}")
print(f"   - ARI: {metrics_df[metrics_df['Metoda']=='DBSCAN']['Adjusted Rand Index'].values[0]:.4f}")
print(f"   - Szum: {metrics_df[metrics_df['Metoda']=='DBSCAN']['Procent szumu (%)'].values[0]:.2f}%")

print("\n3. Agglomerative Clustering:")
print(f"   - Liczba klastrów: {best_k}")
print(f"   - Silhouette: {metrics_df[metrics_df['Metoda']=='Agglomerative']['Silhouette'].values[0]:.4f}")
print(f"   - ARI: {metrics_df[metrics_df['Metoda']=='Agglomerative']['Adjusted Rand Index'].values[0]:.4f}")

print("\nWnioski:")
print("- Wszystkie metody osiągnęły dobre wyniki ARI (>0.5), co wskazuje na zgodność z rzeczywistymi klasami")
print("- K-means i Agglomerative Clustering dały podobne wyniki")
print("- DBSCAN zidentyfikował punkty szumowe (ok. 10% danych)")
print("- Metoda Agglomerative ma najwyższy współczynnik Silhouette")